In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Dokładnia detekcja cyfr z obrazków


### Zadanie: Klasyfikacja i Regresja w Detekcji Obiektów

Twoim zadaniem jest stworzenie modelu sieci neuronowej do detekcji obiektów, który łączy w sobie zarówno zadania klasyfikacji, jak i regresji. Zaprojektowana sieć będzie przyjmować obrazy o wymiarach `(input_size, input_size, 1)` i wykonywać następujące operacje:

1. **Ekstrakcja cech**: Zaimplementowana funkcja `build_feature_extractor(inputs)` przyjmuje obrazy jako wejście i przeprowadza ekstrakcję cech, wykorzystując warstwy konwolucyjne i warstwy max-pooling.

2. **Model Adaptor**: Funkcja `build_model_adaptor(inputs)` przyjmuje wyekstrahowane cechy i przekształca je na jednowymiarowy wektor za pomocą warstw `Flatten` i `Dense`.

3. **Klasyfikacja**: Funkcja `build_classifier_head(inputs)` przyjmuje wyjście z modelu adaptora i dokonuje klasyfikacji na określoną liczbę klas (parametr `n_classes`) za pomocą warstwy `Dense` i funkcji aktywacji softmax.

4. **Regresja**: Funkcja `build_regressor_head(inputs)` również przyjmuje wyjście z modelu adaptora, ale w tym przypadku przewiduje wartości numeryczne, wykorzystując warstwę `Dense` bez funkcji aktywacji.

5. Trening: Wytrenuj model na danych z datasetu mnist-for-object detection. 

Twoim zadaniem jest zaimplementować funkcję `build_object_detection_model(inputs)`, która złoży te części razem w jednym modelu. Model ten będzie miał dwie główne gałęzie wyjściowe: jedna do klasyfikacji obiektów i druga do regresji. Należy także dostosować proces uczenia, określić optymalizator, funkcje straty i metryki.

Po zdefiniowaniu modelu, użyj funkcji `model.compile` do kompilacji modelu z odpowiednimi parametrami.

Podsumowując, Twoje zadanie polega na zaimplementowaniu funkcji `build_object_detection_model(inputs)` i przystosowaniu modelu do jednoczesnej klasyfikacji i regresji w zadaniu detekcji obiektów.

In [ ]:
import numpy as np

# Loading data from the 'mnist_compressed.npz' file
data = np.load('/kaggle/input/mnist-for-object-detection/mnist_object.npz')

# Reading variables containing the data
X_train, y_train, y_train_bboxes =  data['train_images'], data['train_labels'], data['train_bboxes']
X_test, y_test, y_test_bboxes  = data['test_images'], data['test_labels'], data['test_bboxes']

In [ ]:
import matplotlib.pyplot as plt
# Set up the subplot
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i in range(5):
    image = X_test[i]

    # Plot the image and bounding box on the current subplot
    axes[i].imshow(image, cmap='gray')
    axes[i].plot([y_test_bboxes[i, 0], y_test_bboxes[i, 1]], [y_test_bboxes[i, 2], y_test_bboxes[i, 2]], 'r-')
    axes[i].plot([y_test_bboxes[i, 0], y_test_bboxes[i, 1]], [y_test_bboxes[i, 3], y_test_bboxes[i, 3]], 'r-')
    axes[i].plot([y_test_bboxes[i, 0], y_test_bboxes[i, 0]], [y_test_bboxes[i, 2], y_test_bboxes[i, 3]], 'r-')
    axes[i].plot([y_test_bboxes[i, 1], y_test_bboxes[i, 1]], [y_test_bboxes[i, 2], y_test_bboxes[i, 3]], 'r-')
    axes[i].set_title(f"Sample Image {i+1}")

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = 10 # Assuming there are 10 classes (digits from 0 to 9)
y_train = to_categorical(y_train, num_classes)
X_train = np.expand_dims(X_train, axis=-1)

In [ ]:
def build_feature_extractor(inputs):
    layer = Conv2D(8, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    layer = MaxPooling2D(2, 2)(layer)

    layer = Conv2D(32, kernel_size=3, activation='relu')(layer)
    layer = MaxPooling2D(2, 2)(layer)

    layer = Conv2D(64, kernel_size=3, activation='relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = MaxPooling2D(2, 2)(layer)
    return layer

def build_model_adaptor(inputs):
    layer = Flatten()(inputs)
    layer = Dense(128, activation='relu')(layer)
    layer = Dense(64, activation='relu')(layer)
    return layer


def build_classifier_head(inputs):
    return Dense(n_classes, activation='softmax', name='classifier_head')(inputs)

def build_regressor_head(inputs):
    return Dense(units=4, name='regressor_head')(inputs)

def build_object_detection_model(inputs):
    feature_extractor = build_feature_extractor(inputs)

    model_adaptor = build_model_adaptor(feature_extractor)
    classification_head = build_classifier_head(model_adaptor)
    regressor_head = build_regressor_head(model_adaptor)

    model = Model(inputs=inputs, outputs=[classification_head, regressor_head])

    model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss={'classifier_head': 'categorical_crossentropy', 'regressor_head': 'mse'},
    metrics={'classifier_head': 'accuracy', 'regressor_head': 'mse'}
    )

    return model




In [ ]:
#todo model 2 głowicowy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
input_size=28
n_classes=10
model=build_object_detection_model(Input(shape=(input_size, input_size,1)))
model.summary()

In [ ]:
model.fit(X_train, [y_train, y_train_bboxes], epochs=20, batch_size=1000)
y_pred,y_regres_pred=model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
# Set up the subplot
#fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i in range(5):
    image = X_test[i]

    plt.figure(figsize = (20,2))
    plt.imshow(image, cmap='gray')
    plt.plot([y_regres_pred[i, 0], y_regres_pred[i, 1]], [y_regres_pred[i, 2], y_regres_pred[i, 2]], 'r-')
    plt.plot([y_regres_pred[i, 0], y_regres_pred[i, 1]], [y_regres_pred[i, 3], y_regres_pred[i, 3]], 'r-')
    plt.plot([y_regres_pred[i, 0], y_regres_pred[i, 0]], [y_regres_pred[i, 2], y_regres_pred[i, 3]], 'r-')
    plt.plot([y_regres_pred[i, 1], y_regres_pred[i, 1]], [y_regres_pred[i, 2], y_regres_pred[i, 3]], 'r-')
    plt.show()


Oto treść zadania opartego na podanym kodzie:

### Zadanie: Analiza sentymentu tekstów za pomocą modelu LSTM

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie przewidywać sentyment na podstawie tekstu z wykorzystaniem modelu LSTM. Dane tekstowe są dostarczone w pliku CSV, który został już wczytany do ramki danych `df`.

Kroki, które musisz podjąć:

1. **Przygotowanie danych**: Dane są wczytane z pliku CSV, a etykiety sentymentu są zamienione na wartości liczbowe (0 dla "negative", 1 dla "positive" i 2 dla "neutral"). Teksty są przekształcane na sekwencje liczb za pomocą Tokenizera. Następnie dane są podzielone na zbiór treningowy, walidacyjny i testowy.

2. **Model LSTM**: Model jest zdefiniowany w kodzie i składa się z następujących warstw:
   - Warstwa Embedding: Mapuje słowa na wektory o określonej długości.
   - Warstwa LSTM: Warstwa rekurencyjna LSTM, która przetwarza sekwencje wektorów.
   - Warstwa Dense: Warstwa wyjściowa z funkcją aktywacji softmax, która generuje trzy możliwe klasy sentymentu.

3. **Kompilacja modelu**: Model jest skompilowany z odpowiednią funkcją straty "binary_crossentropy", optymalizatorem "adam" i metryką "accuracy".

4. **Trenowanie modelu**: Model jest trenowany na danych treningowych przez 10 epok, z wsadem o rozmiarze 10. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie tego kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy sentymentu tekstu. Możesz eksperymentować z parametrami, takimi jak rozmiar sekwencji (max_sequence_length), wymiar embeddingu (embedding_dim) czy liczba epok treningu, aby dostosować model do danego zadania analizy sentymentu.

In [1]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/financial-sentiment-analysis/data.csv')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [2]:
df['Sentiment'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [3]:
# Mapowanie etykiet sentymentu na wartości float
sentiment_map = {"positive": 1, "negative": 0, 'neutral': 2}
df['Sentiment'] = df['Sentiment'].map(sentiment_map)


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [10]:
def clean_sentence_enhanced_v2(sentence):
   # Konwersja na małe litery
   sentence = sentence.lower()
   # Usuwanie kwot z symbolem dolara i liczb z przecinkami
   sentence = re.sub(r'\$\d+(?:\.\d+)?', '', sentence)
   sentence = re.sub(r'\d+,\d+', '', sentence)
   # Zamiana URLi
   sentence = re.sub(r'http\S+|www\S+|https\S+', '<URL>', sentence, flags=re.MULTILINE)
   # Zamiana wzmianek o użytkownikach
   sentence = re.sub(r'@\w+', '<USER>', sentence)
   # Zamiana znaków '#'
   sentence = re.sub(r'#', '<TAG>', sentence)
   # Zamiana pozostałych cyfr
   sentence = re.sub(r'\d+', '<NUMBER>', sentence)
   # Usuwanie znaków interpunkcyjnych
   sentence = ''.join(ch for ch in sentence if ch not in string.punctuation)
   return sentence


In [12]:
import re
import string
from tensorflow.keras.utils import to_categorical
X = df['Sentence']
print(X.head())
y = df['Sentiment']
X=X.apply(clean_sentence_enhanced_v2)
print(X.head())
num_classes = 3 
y = to_categorical(y, num_classes)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

0    The GeoSolutions technology will leverage Bene...
1    $ESI on lows, down $1.50 to $2.50 BK a real po...
2    For the last quarter of 2010 , Componenta 's n...
3    According to the Finnish-Russian Chamber of Co...
4    The Swedish buyout firm has sold its remaining...
Name: Sentence, dtype: object
0    the geosolutions technology will leverage bene...
1          esi on lows down  to  bk a real possibility
2    for the last quarter of NUMBER  componenta s n...
3    according to the finnishrussian chamber of com...
4    the swedish buyout firm has sold its remaining...
Name: Sentence, dtype: object


In [18]:
df['Sentence'].shape

(5842,)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
max_sequence_length = max(len(seq) for seq in X_train)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)



In [ ]:
np.max(X_val)



In [ ]:
#todo model i validacja
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
model = Sequential()
model.add(Embedding(input_dim=10648, output_dim=52))
model.add(LSTM(52, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=100, verbose=1, validation_data=(X_val,y_val))


### Zadanie: Klasyfikacja kotów i psów za pomocą modelu ResNet50

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować obrazy kotów i psów. W tym celu wykorzystaj model ResNet50, który został wstępnie przeszkolony na szerokim zbiorze danych.

Kroki, które musisz podjąć:

1. **Załadowanie danych**: Dane treningowe i walidacyjne zostały już przygotowane w formie generatorów za pomocą biblioteki `ImageDataGenerator`. Generator treningowy zawiera obrazy treningowe, a generator walidacyjny zawiera obrazy walidacyjne. Obrazy są przeskalowane do rozmiaru `(224, 224)` i podzielone na dwie klasy: koty i psy.

2. **Model ResNet50**: W kodzie załadowano model ResNet50, który jest wstępnie przeszkolony na zestawie danych ImageNet. Model ten ma zamrożone wagi wszystkich warstw.

3. **Warstwy klasyfikacji**: Do modelu ResNet50 dodano warstwy klasyfikacji. Jest to warstwa `GlobalAveragePooling2D` i warstwa `Dense`, która wykorzystuje funkcję aktywacji sigmoid. Model ten jest przystosowany do zadania klasyfikacji binarnej, gdzie klasy to "kot" i "pies".

4. **Kompilacja modelu**: Model jest skompilowany z optymalizatorem Adam, funkcją straty "binary_crossentropy" (odpowiednia do klasyfikacji binarnej), oraz metryką "accuracy".

5. **Trenowanie modelu**: Model jest trenowany na danych treningowych przez jedną epokę. Wynik trenowania jest wyświetlany na konsoli.

Twoim zadaniem jest uzupełnienie tego kodu o brakujące elementy, takie jak liczba epok trenowania, odblokowanie warstw modelu ResNet50 oraz dostosowanie modelu do odpowiedniego rodzaju klasyfikacji (binarna). Następnie możesz dostosować parametry trenowania, takie jak liczba epok czy wielkość wsadu (batch size), aby osiągnąć jak najlepsze wyniki w zadaniu klasyfikacji kotów i psów.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie ścieżki do katalogu z danymi
data_dir = '/kaggle/input/cat-and-dog/'

# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255,           # Przeskalowanie wartości pikseli do zakresu [0, 1]
    validation_split=0.2       # Procent danych przeznaczonych na zbiór walidacyjny
)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=32,             # Rozmiar wsadu
    class_mode='binary',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='training'          # Ustawienie dla danych treningowych
)

# Wczytanie danych walidacyjnych
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'        # Ustawienie dla danych walidacyjnych
)

Found 8023 images belonging to 2 classes.
Found 2005 images belonging to 2 classes.


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam



In [5]:
# Sprawdź liczbę klas
num_classes = len(train_generator.class_indices)
print("Liczba klas:", num_classes)

# Sprawdź rozmiar pierwszej partii danych treningowych
batch_images, batch_labels = train_generator.next()
print("Rozmiar partii obrazów:", batch_images.shape)
print("Rozmiar partii etykiet:", batch_labels.shape)

Liczba klas: 2
Rozmiar partii obrazów: (32, 224, 224, 3)
Rozmiar partii etykiet: (32,)


In [4]:
num_epochs = 1
output_logistic_regression = 1
# Loading the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing all layers of the ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Adding new classification layers
model = Sequential([
   base_model,
   GlobalAveragePooling2D(),
   Dense(output_logistic_regression, activation='sigmoid') # num_classes is the number of classes
])

# Compiling the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=1,                 # Liczba epok treningu
    validation_data=validation_generator)


  3/251 [..............................] - ETA: 14:33 - loss: 1.4385 - accuracy: 0.1562

KeyboardInterrupt: 